In [1]:
import numpy as np
import pandas as pd
import json
# haandle data
import json
import operator
import collections
from sklearn import svm
from sklearn.model_selection  import train_test_split

In [2]:
# load json file
with open('./data/train.json') as data_file:
    train_data = json.load(data_file)

with open('./data/test.json') as data_file:
    test_data = json.load(data_file)

In [3]:
# get all ingredients in all dishes
def get_ingredients(json):
    raw_ingredients = list()
    for dish in json:
        for ingredient in dish['ingredients']:
            raw_ingredients.append(ingredient.strip())
    return raw_ingredients
raw_ingredients = list(set(get_ingredients(train_data)))
train_ingredients = collections.Counter(get_ingredients(train_data))
# get all cusines

def get_cuisine(json):
    raw_cuisines = list()
    for dish in json:
        raw_cuisines.append(dish['cuisine'].strip())
    return raw_cuisines
raw_cuisines = list(set(get_cuisine(train_data)))
train_cuisines =  collections.Counter(get_cuisine(train_data))

In [4]:
# print(train_data[:10])

In [5]:
number_of_ingredients = len(train_ingredients)
print("number of ingredients:", number_of_ingredients)
number_of_train_dishes = len(train_data)
print("number of trainning dishes:", number_of_train_dishes )
number_of_test_dishes = len(test_data)
print("number of testing dishes:", number_of_test_dishes)

number of ingredients: 6714
number of trainning dishes: 39774
number of testing dishes: 9944


In [21]:
model_matrix = np.zeros((number_of_train_dishes, number_of_ingredients))
for row, dish in enumerate(train_data):
    ingredients = dish['ingredients']
    for ingredient in ingredients:
        col = raw_ingredients.index(ingredient)
        model_matrix[row, col] = 1
        
def create_model_matrix(json):
    data_size = len(json)
    model_matrix = np.zeros((data_size, number_of_ingredients))
    for row, dish in enumerate(json):
        ingredients = dish['ingredients']
        for ingredient in ingredients:
            if ingredient in raw_ingredients:
                col = raw_ingredients.index(ingredient)
                model_matrix[row, col] = 1
    return model_matrix

test_model_matrix = create_model_matrix(test_data)

In [7]:
print('features:', len(model_matrix[0]))
print('data:', print(len(model_matrix)))

features: 6714
39774
data: None


In [8]:
print('labels:',  raw_cuisines, ":", len(raw_cuisines))
print('labels data:',  len([raw_cuisines.index(dish['cuisine']) for dish in train_data]))

labels: ['mexican', 'british', 'brazilian', 'greek', 'jamaican', 'southern_us', 'korean', 'spanish', 'indian', 'russian', 'chinese', 'cajun_creole', 'irish', 'vietnamese', 'thai', 'moroccan', 'japanese', 'filipino', 'italian', 'french'] : 20
labels data: 39774


In [9]:
features = model_matrix


from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
# make label into onehot array
raw_target = [raw_cuisines.index(dish['cuisine']) for dish in train_data]
one_hot_label = to_categorical(raw_target)
target = one_hot_label

Using TensorFlow backend.


In [10]:
train_X, test_X, train_y, test_y = train_test_split(features, target, random_state=0)

In [82]:
print(len(one_hot_label[0]))
print(len(one_hot_label))

20
39774


In [70]:
print(train_X[0])
print(one_hot_label[0])
raw_cuisines[np.argmax(one_hot_label)]

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


'greek'

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
clf = KNeighborsRegressor()
clf.fit(train_X, train_y)
print(clf.score(test_X, test_y))

In [48]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [50]:
# define model 
model = Sequential()
model.add(Dense(300, input_shape=(6714,)))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dropout(0.25))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dense(300, activation=tf.nn.relu))
model.add(Dense(20, activation=tf.nn.softmax))

In [51]:
model.compile(optimizer='adam', 
             loss = 'categorical_crossentropy', 
             metrics=['accuracy'])

In [53]:
batchsize = 100
epochs = 20
model.fit(train_X, train_y, epochs=epochs, batch_size=batchsize, validation_data=(test_X, test_y))

Train on 29830 samples, validate on 9944 samples
Epoch 1/20
29830/29830 [==============================] - 19s 634us/step - loss: 1.3796 - acc: 0.5889 - val_loss: 1.0108 - val_acc: 0.7039
Epoch 2/20
29830/29830 [==============================] - 19s 624us/step - loss: 0.8166 - acc: 0.7567 - val_loss: 0.9232 - val_acc: 0.7374
Epoch 3/20
29830/29830 [==============================] - 19s 623us/step - loss: 0.6224 - acc: 0.8162 - val_loss: 0.8851 - val_acc: 0.7543
Epoch 4/20
29830/29830 [==============================] - 20s 668us/step - loss: 0.4940 - acc: 0.8545 - val_loss: 0.9142 - val_acc: 0.7566
Epoch 5/20
29830/29830 [==============================] - 19s 652us/step - loss: 0.3986 - acc: 0.8812 - val_loss: 0.9779 - val_acc: 0.7555
Epoch 6/20
29830/29830 [==============================] - 18s 593us/step - loss: 0.3221 - acc: 0.9031 - val_loss: 1.0346 - val_acc: 0.7603
Epoch 7/20
29830/29830 [==============================] - 18s 591us/step - loss: 0.2667 - acc: 0.9215 - val_loss: 1.1

In [41]:
model.save('./whats_cooking_ingredients_as_features.model')

# Submit model

In [54]:
predictions = model.predict(test_model_matrix)

In [55]:
ss = pd.read_csv('./data/sample_submission.csv')
ss.head()

,id,cuisine
0,35203,italian
1,17600,italian
2,35200,italian
3,17602,italian
4,17605,italian


In [56]:
written_preds = [raw_cuisines[np.argmax(pred)] for pred in predictions]

In [57]:
ss['cuisine'] = written_preds

In [58]:
ss.to_csv('./data/submission.csv', index=False)